## Cleaning & Standardizing 10k Iranian Dirty Transactions Dataset

In [2]:
import pandas as pd


df = pd.read_csv('archive/trx-10k.csv')


print("--- 5 Baris Pertama ---")
display(df.head())


print("\n--- Info Tipe Data ---")
print(df.info())

--- 5 Baris Pertama ---


,status,time,card_type,city,amount,id
0,success,2025-09-07 10:48:00,Visa,Tehran,1427657.0,98
1,success,2025-09-20 13:10:00,MastCard,Tehran,1578078.0,52
2,fail,2025-09-29 03:27:00,Visa,Tehran,250000.0,20
3,success,2025-09-15 13:13:00,Discover,Isfahan,0.0,40
4,success,2025-09-11 11:11:00,Visa,Tehran,1782689.0,59



--- Info Tipe Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   status     10000 non-null  object 
 1   time       10000 non-null  object 
 2   card_type  9689 non-null   object 
 3   city       9884 non-null   object 
 4   amount     10000 non-null  float64
 5   id         10000 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB
None


**Handling missing values**

In [3]:
print(df.isna().sum())

status         0
time           0
card_type    311
city         116
amount         0
id             0
dtype: int64


In [4]:
# # Hitung jumlah null
# null_counts = df.isnull().sum()

# # Hitung persentase
# null_pct = (null_counts / len(df)) * 100

# # Gabungkan jadi dataframe biar enak dilihat
# null_summary = pd.DataFrame({
#     'Jumlah Null': null_counts,
#     'Persentase (%)': null_pct
# })

# # Tampilkan hanya kolom yang ada null-nya, urutkan dari yang terbesar
# print("\n--- Summary Missing Values ---")
# print(null_summary[null_summary['Jumlah Null'] > 0].sort_values('Persentase (%)', ascending=False))

In [5]:
threshold = len(df) * 0.05 
print(threshold)

500.0


In [6]:
cols_to_drop = df.columns[df.isna().sum() <= threshold] 
print(cols_to_drop)

Index(['status', 'time', 'card_type', 'city', 'amount', 'id'], dtype='object')


In [7]:
df.dropna(subset=cols_to_drop, inplace=True)

In [8]:
print(df.isna().sum())

status       0
time         0
card_type    0
city         0
amount       0
id           0
dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9578 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   status     9578 non-null   object 
 1   time       9578 non-null   object 
 2   card_type  9578 non-null   object 
 3   city       9578 non-null   object 
 4   amount     9578 non-null   float64
 5   id         9578 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 523.8+ KB


## Data Standardization / Mapping

***status**

In [10]:
print(df['status'].value_counts())

status
fail       3792
success    3761
failed     1133
Success     475
FAIL        212
succeed     205
Name: count, dtype: int64


In [11]:
df['status'].unique()

array(['success', 'fail', 'Success', 'failed', 'succeed', 'FAIL'],
      dtype=object)

In [12]:
df['status'] = df['status'].astype(str).str.lower().str.strip()


clean_map = {
    'success': 'Success',
    'succeed': 'Success',
    'fail': 'Fail',
    'failed': 'Fail'
}


df['status'] = df['status'].map(clean_map).astype('category')


print("Nilai Unique:", df['status'].unique())
print("\nJumlah per Kategori:")
print(df['status'].value_counts())


Nilai Unique: ['Success', 'Fail']
Categories (2, object): ['Fail', 'Success']

Jumlah per Kategori:
status
Fail       5137
Success    4441
Name: count, dtype: int64


In [13]:
# df['status'] = df['status'].astype(str)

# def filter_status(val):
#     v = val.lower().strip() # kecilin & hapus spasi liar
#     if 'succ' in v:
#         return 'Success'
#     else:
#         return 'Fail'


# df['status'] = df['status'].apply(filter_status)


# df['status'] = df['status'].astype('category')


# print(df['status'].unique())
# print(df['status'].value_counts())


***time***

In [14]:
df['time'] =  pd.to_datetime(df['time'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9578 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   status     9578 non-null   category      
 1   time       9578 non-null   datetime64[ns]
 2   card_type  9578 non-null   object        
 3   city       9578 non-null   object        
 4   amount     9578 non-null   float64       
 5   id         9578 non-null   int64         
dtypes: category(1), datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 458.4+ KB


***card_type***

In [16]:
print(df['card_type'].value_counts())

card_type
Visa           2336
Discover       1632
MasterCard     1425
Amex           1248
MastCard        988
Vsa             792
Master-Card     586
visa            205
Master Card     189
VISA            177
Name: count, dtype: int64


In [17]:
print(df['card_type'].unique())

['Visa' 'MastCard' 'Discover' 'Master-Card' 'Amex' 'visa' 'MasterCard'
 'Vsa' 'VISA' 'Master Card']


In [18]:
df['card_type'] = df['card_type'].astype(str).str.lower().str.strip()

cc_mapping = {
    'visa': 'Visa',
    'vsa': 'Visa', 
    'mastcard': 'MasterCard', 
    'master-card': 'MasterCard', 
    'mastercard': 'MasterCard', 
    'master card': 'MasterCard',
    'amex': 'Amex', 
    'discover': 'Discover'
}

df['card_type'] = df['card_type'].map(cc_mapping).astype('category')

print("Nilai Unique:", df['card_type'].unique())
print("\nJumlah per Kategori:")
print(df['card_type'].value_counts())
print("Total_rows:", df['card_type'].count())
len(df)


Nilai Unique: ['Visa', 'MasterCard', 'Discover', 'Amex']
Categories (4, object): ['Amex', 'Discover', 'MasterCard', 'Visa']

Jumlah per Kategori:
card_type
Visa          3510
MasterCard    3188
Discover      1632
Amex          1248
Name: count, dtype: int64
Total_rows: 9578


9578

***City***

In [19]:
print(df['city'].value_counts())

city
Tehran     2003
Tabriz     1325
Isfahan    1037
Mashhad     875
Shiraz      706
Qom         646
Karaj       624
Ahvaz       611
TEHRAN      393
THR         390
karaj       386
tehr@n      321
Tehran      185
ThRan        76
Name: count, dtype: int64


In [20]:
print(df['city'].unique())

['Tehran' 'Tehran ' 'Isfahan' 'Tabriz' 'Shiraz' 'TEHRAN' 'karaj' 'Ahvaz'
 'THR' 'Qom' 'Mashhad' 'tehr@n' 'Karaj' 'ThRan']


In [21]:
df['city'] = df['city'].astype(str).str.lower().str.strip()


# sisain huruf a-z
df['city'] = df['city'].str.replace(r'[^a-z]', '', regex=True)


city_map = {
    'tehran': 'Tehran',
    'thr': 'Tehran',
    'tehrn': 'Tehran', # Hasil dari tehr@n setelah simbol dihapus
    'thran': 'Tehran',
    'tabriz': 'Tabriz',
    'isfahan': 'Isfahan',
    'mashhad': 'Mashhad',
    'shiraz': 'Shiraz',
    'qom': 'Qom',
    'karaj': 'Karaj',
    'ahvaz': 'Ahvaz'
}


df['city'] = df['city'].map(city_map).astype('category')

if df['city'].isna().any():
    print("Waduh, ada yang ketinggalan nih di kamus!")


print(df['city'].value_counts())
print("Total_rows:", df['city'].count())
print(len(df))


city
Tehran     3368
Tabriz     1325
Isfahan    1037
Karaj      1010
Mashhad     875
Shiraz      706
Qom         646
Ahvaz       611
Name: count, dtype: int64
Total_rows: 9578
9578


**amount**

In [22]:
print(df['amount'].describe())

count    9.578000e+03
mean     2.959419e+08
std      1.693342e+09
min     -9.999990e+05
25%      1.005000e+02
50%      3.436790e+05
75%      9.144650e+05
max      1.000000e+10
Name: amount, dtype: float64


In [23]:
total_nol = (df['amount'] == 0).sum()
print(f"Ada {total_nol} baris yang isinya 0")

Ada 941 baris yang isinya 0


In [24]:
# Lihat 10 nilai terkecil yang unik
print(df['amount'].sort_values().unique()[:10])


[-9.99999e+05 -5.00000e+03 -1.00000e+00  0.00000e+00  1.00000e+00
  1.00500e+02  1.00000e+03  5.00000e+03  1.23450e+04  5.02000e+04]


In [25]:
Q1 = df['amount'].quantile(0.25)
Q3 = df['amount'].quantile(0.75)
IQR = Q3 - Q1

#  3 * IQR (Extreme Outlier)
batas_atas = Q3 + (3 * IQR)

outliers = df[df['amount'] > batas_atas]

print(f"Batas Atas: {batas_atas:,.2f}")
print(f"Jumlah Outlier yang Terdeteksi: {len(outliers)}")
print("\nContoh data yang terdeteksi sebagai Outlier:")
print(outliers[['city', 'card_type', 'amount']].sort_values(by='amount', ascending=False).head(10))


Batas Atas: 3,657,558.50
Jumlah Outlier yang Terdeteksi: 283

Contoh data yang terdeteksi sebagai Outlier:
        city   card_type        amount
24    Shiraz        Visa  1.000000e+10
35    Tehran        Visa  1.000000e+10
53     Ahvaz  MasterCard  1.000000e+10
61   Isfahan        Visa  1.000000e+10
124   Tabriz        Visa  1.000000e+10
133      Qom    Discover  1.000000e+10
199  Isfahan        Amex  1.000000e+10
204   Tehran    Discover  1.000000e+10
215    Karaj        Amex  1.000000e+10
234  Mashhad        Visa  1.000000e+10


In [26]:
import numpy as np

# Ubah yang <= 0 jadi NaN 
df.loc[df['amount'] <= 0, 'amount'] = np.nan


val_median = df['amount'].median()


df['amount'] = df['amount'].fillna(val_median)


print(f"Total rows: {len(df)}")
print(f"Nilai Min: {df['amount'].min()}")
print(f"Cek NaN: {df['amount'].isna().sum()}")

Total rows: 9578
Nilai Min: 1.0
Cek NaN: 0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9578 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   status     9578 non-null   category      
 1   time       9578 non-null   datetime64[ns]
 2   card_type  9578 non-null   category      
 3   city       9578 non-null   category      
 4   amount     9578 non-null   float64       
 5   id         9578 non-null   int64         
dtypes: category(3), datetime64[ns](1), float64(1), int64(1)
memory usage: 328.0 KB


In [28]:
print(df.head())

    status                time   card_type     city     amount  id
0  Success 2025-09-07 10:48:00        Visa   Tehran  1427657.0  98
1  Success 2025-09-20 13:10:00  MasterCard   Tehran  1578078.0  52
2     Fail 2025-09-29 03:27:00        Visa   Tehran   250000.0  20
3  Success 2025-09-15 13:13:00    Discover  Isfahan   555555.0  40
4  Success 2025-09-11 11:11:00        Visa   Tehran  1782689.0  59


In [29]:
df.to_csv('cleaned_data.csv', index=False)

print("Data berhasil disimpan ke 'cleaned_data.csv'")

Data berhasil disimpan ke 'cleaned_data.csv'


- First of all, I handled a messy dataset with 10,000 rows.
- During the process, I found some issues like inconsistent city names, and zero values in the amount column.
- To fix this, I used Regex to clean the symbols and applied Median Imputation to handle zero and negative values because the outliers were too extreme.
- Finally, after cleaning, I managed to keep 9,578 rows with consistent data types.

In [30]:
# --- STEP TAMBAHAN: Handling Extreme Outliers ---

# Kita pakai batas_atas yang tadi sudah kamu hitung (Q3 + 3*IQR)
# Q1 = df['amount'].quantile(0.25)
# Q3 = df['amount'].quantile(0.75)
# IQR = Q3 - Q1
# batas_atas = Q3 + (3 * IQR)

print(f"Membuang transaksi di atas: {batas_atas:,.0f}")

# Cek jumlah baris SEBELUM dibuang
print(f"Total baris sebelum drop outlier: {len(df)}")

# EKSEKUSI: Hanya ambil data yang KURANG DARI atau SAMA DENGAN batas_atas
df = df[df['amount'] <= batas_atas]

# Cek jumlah baris SETELAH dibuang
print(f"Total baris setelah drop outlier: {len(df)}")

# Cek statistik akhir (Max-nya harusnya tidak lagi 10 Miliar)
print(df['amount'].describe())

Membuang transaksi di atas: 3,657,558
Total baris sebelum drop outlier: 9578
Total baris setelah drop outlier: 9295
count    9.295000e+03
mean     6.534302e+05
std      5.000137e+05
min      1.000000e+00
25%      2.500000e+05
50%      5.555550e+05
75%      9.142820e+05
max      1.883871e+06
Name: amount, dtype: float64
